In [1]:
from scipy.optimize import minimize
import pandas as pd
import numpy as np

In [3]:

# Initialize the table of parallelism score for both model and human ratings
wordseg = pd.read_csv('word_segment.csv')
pos = pd.read_csv('pos_survey_ouput.csv')
sem = pd.read_csv('semantic.csv')
sem2 = pd.read_excel('semantic2.xlsx')
sim = pd.DataFrame({'Word Seg': wordseg['cosine_similarity'],
                    'POS Tagging': pos['Cosine_similarity'],
                    'Sem: Sentence': sem2['Similarity'],
                    'Sem: Character': sem['Similarity']})
human = np.array([8.092715, 6.741722, 7.298013, 8.470199, 7.523179, 7.470199, 8.410596, 9.013245])
sim['Human Rating'] = human

# Initialize function and parameters for optimzation
w_initial = np.array([1/4, 1/4, 1/4, 1/4])
model_output = sim[['Sem: Character','Sem: Sentence' ,'POS Tagging', 'Word Seg']]*10
def rmse(w):
    model = model_output @ w 
    return np.sqrt(np.mean((human - model) ** 2))

# Constraints: weights sum to 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Bounds: weights are between 0 and 1
bounds = [(0, 1) for _ in range(len(w_initial))]

# Minimize RMSE
result = minimize(rmse, w_initial, constraints=constraints, bounds=bounds)

# Optimized weights
w_optimized = result.x

print("Optimized weights:", w_optimized)
print("Minimized RMSE:", rmse(w_optimized))


sim['weighted_ave*10'] = model_output @ w_optimized
display(sim)



Optimized weights: [0.39889944 0.14893276 0.20507788 0.24708993]
Minimized RMSE: 0.2841199363810805


,Word Seg,POS Tagging,Sem: Sentence,Sem: Character,Human Rating,weighted_ave*10
0,0.816497,0.990683,0.766967,0.752056,8.092715,8.191366
1,0.471405,0.771589,0.721520,0.663895,6.741722,6.470003
2,0.925820,0.611607,0.752221,0.618250,7.298013,7.128377
3,1.000000,0.989565,0.894446,0.731698,8.470199,8.751142
4,0.833333,0.665038,0.757544,0.661324,7.523179,7.189178
5,0.912871,0.770374,0.704952,0.617138,7.470199,7.347144
6,1.000000,0.937875,1.000000,0.701200,8.410596,8.680684
7,0.894427,0.979158,0.850157,0.758124,9.013245,8.508391
